In [ ]:
MyAPI = 'Replace Youtube API Key'
MyID = "Replace with youtube video URL"

In [ ]:
# prompt: write a program to extract these informations "VideoID	Channel	NumOfCommentlikes	NumOfCommentDislikes	NumOfReplies	Comment	CommentedUserID	RepliedUserID	Reply	RepliesLikes	RepliesDislike	ToWhomTheyReplied	Comment|repliedTime	VideoUploadedtimeanddate" using youtube api and save the file as csv file as videoid as file name.

from googleapiclient.discovery import build
import csv
#import isodate

# Replace with your API key and video ID
#MyAPI = 'YOUR_API_KEY' 
#MyID = 'YOUR_VIDEO_ID' 

!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

api_key = MyAPI
youtube = build('youtube', 'v3', developerKey=api_key)

def get_video_details(video_id, api_key):
  request = youtube.videos().list(
      part="snippet,statistics",
      id=video_id
  )
  response = request.execute()
  return response['items'][0]

def get_comments(video_id, api_key):
  comments = []

  request = youtube.commentThreads().list(
      part="snippet,replies",
      videoId=video_id,
      textFormat="plainText"
  )

  while request:
    response = request.execute()

    for item in response['items']:
      comment = item['snippet']['topLevelComment']['snippet']
      comment_data = {
          'VideoID': video_id,
          'Channel': comment['authorDisplayName'],
          'NumOfCommentlikes': comment['likeCount'],
          'NumOfCommentDislikes': comment.get('dislikeCount', 0), # Dislikes are not available anymore
          'Comment': comment['textDisplay'],
          'CommentedUserID': comment['authorChannelUrl'],
          'Comment|repliedTime': comment['publishedAt'], 
      }

      if 'replies' in item:
        for reply in item['replies']['comments']:
          reply_snippet = reply['snippet']
          comment_data['NumOfReplies'] = len(item['replies']['comments'])
          comment_data['RepliedUserID'] = reply_snippet['authorChannelUrl']
          comment_data['Reply'] = reply_snippet['textDisplay']
          comment_data['RepliesLikes'] = reply_snippet['likeCount']
          comment_data['RepliesDislike'] = reply_snippet.get('dislikeCount', 0) # Dislikes are not available anymore
          comment_data['ToWhomTheyReplied'] = comment['authorDisplayName']
          comments.append(comment_data.copy())
      else:
        comment_data['NumOfReplies'] = 0
        comment_data['RepliedUserID'] = ''
        comment_data['Reply'] = ''
        comment_data['RepliesLikes'] = 0
        comment_data['RepliesDislike'] = 0
        comment_data['ToWhomTheyReplied'] = ''
        comments.append(comment_data.copy())

    request = youtube.commentThreads().list_next(request, response)

  return comments

video_details = get_video_details(MyID, api_key)
comments = get_comments(MyID, api_key)

# Get video upload time and date
video_upload_time = video_details['snippet']['publishedAt']

# Add video upload time to each comment
for comment in comments:
  comment['VideoUploadedtimeanddate'] = video_upload_time

filename = f"{MyID}.csv"
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
  fieldnames = ['VideoID', 'Channel', 'NumOfCommentlikes', 'NumOfCommentDislikes', 'NumOfReplies', 'Comment', 'CommentedUserID', 'RepliedUserID', 'Reply', 'RepliesLikes', 'RepliesDislike', 'ToWhomTheyReplied', 'Comment|repliedTime', 'VideoUploadedtimeanddate'] 
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
  writer.writeheader()
  for comment in comments:
    writer.writerow(comment)

# Multiple Comments download from ChannelID 

In [ ]:
# prompt: write a code to extract these information from "videoId,title, publishedAt, description, channelTitle, tags, duration, viewCount, likeCount, favoriteCount, TotalVideoCommentsCount, videoId, Channel, NumOfCommentlikes, NumOfReplies, Comment, CommentedUserID, RepliedUserID, Reply, RepliesLikes, ToWhomTheyReplied" using youtube api and save the file as csv file as videoid as file name. Also for future use case check that csv file is existed or not if not create file and save else check the data is presented or not, if TotalVideoCommentsCount is equal to number of comments extracted from the same video then go to next videoId until fill complete information.

!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

from googleapiclient.discovery import build
import csv
import os

# Set up the YouTube Data API v3 service
API_KEY = 'A********************-****************E'  # Replace with your actual API key
youtube = build('youtube', 'v3', developerKey=API_KEY)

def get_channel_videos(channel_id):
    """Retrieve all video IDs from a given channel."""
    video_ids = []
    next_page_token = None

    while True:
        request = youtube.search().list(
            part="id",
            channelId=channel_id,
            maxResults=50,
            pageToken=next_page_token,
            type="video"
        )
        response = request.execute()

        video_ids.extend([item['id']['videoId'] for item in response['items']])

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return video_ids

def get_video_details(video_id):
    """Retrieve video details and comments using the YouTube Data API."""
    video_response = youtube.videos().list(
        part='snippet,contentDetails,statistics',
        id=video_id
    ).execute()

    video_details = {}
    comments_data = []
    if 'items' in video_response and video_response['items']:
        item = video_response['items'][0]
        snippet = item['snippet']
        statistics = item['statistics']
        content_details = item['contentDetails']

        video_details['videoId'] = video_id
        video_details['title'] = snippet.get('title', '')
        video_details['publishedAt'] = snippet.get('publishedAt', '')
        video_details['description'] = snippet.get('description', '')
        video_details['channelTitle'] = snippet.get('channelTitle', '')
        video_details['tags'] = ','.join(snippet.get('tags', []))
        video_details['duration'] = content_details.get('duration', '')
        video_details['viewCount'] = statistics.get('viewCount', 0)
        video_details['likeCount'] = statistics.get('likeCount', 0)
        video_details['favoriteCount'] = statistics.get('favoriteCount', 0)

        # Get top-level comments
        comments_response = youtube.commentThreads().list(
            part='snippet,replies',
            videoId=video_id,
            textFormat='plainText',
            maxResults=100000  # Adjust as needed
        ).execute()

        for item in comments_response.get('items', []):
            comment = item['snippet']['topLevelComment']['snippet']
            comments_data.append({
                'TotalVideoCommentsCount': len(comments_response.get('items', [])),
                'videoId': video_id,
                'Channel': video_details['channelTitle'],
                'NumOfCommentlikes': comment.get('likeCount', 0),
                'Comment': comment.get('textDisplay', ''),
                'CommentedUserID': comment['authorChannelId']['value'],
                'NumOfReplies': item['snippet'].get('totalReplyCount', 0),
            })

            # Get replies (if any)
            if 'replies' in item:
                for reply in item['replies']['comments']:
                    reply_snippet = reply['snippet']
                    comments_data.append({
                        'TotalVideoCommentsCount': len(comments_response.get('items', [])),
                        'videoId': video_id,
                        'Channel': video_details['channelTitle'],
                        'Reply': reply_snippet.get('textDisplay', ''),
                        'RepliesLikes': reply_snippet.get('likeCount', 0),
                        'RepliedUserID': reply_snippet['authorChannelId']['value'],
                        'ToWhomTheyReplied': comment['authorChannelId']['value'],
                    })
        return video_details, comments_data
    else:
        return None, None

def save_to_csv(channel_name, video_details, comments_data):
    """Save video details and comments to a CSV file named after the channel."""
    filename = f'{channel_name}.csv'
    file_exists = os.path.isfile(filename)
    existing_video_ids = set()

    if file_exists:
        with open(filename, 'r', newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            existing_video_ids = {row['videoId'] for row in reader}

    if video_details['videoId'] in existing_video_ids:
        return

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['videoId', 'title', 'publishedAt', 'description', 'channelTitle', 
                      'tags', 'duration', 'viewCount', 'likeCount', 'favoriteCount',
                      'TotalVideoCommentsCount', 'Channel', 'NumOfCommentlikes', 
                      'NumOfReplies', 'Comment', 'CommentedUserID', 'RepliedUserID', 
                      'Reply', 'RepliesLikes', 'ToWhomTheyReplied']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()

        if video_details:
            writer.writerow(video_details)
        if comments_data:
            writer.writerows(comments_data)

def process_channel(channel_id):
    """Process all videos in a channel and save details to CSV."""
    video_ids = get_channel_videos(channel_id)
    for video_id in video_ids:
        video_details, comments_data = get_video_details(video_id)
        
        if video_details and comments_data:
            save_to_csv(video_details['channelTitle'], video_details, comments_data)
        else:
            print(f'Error retrieving video data for video ID {video_id}.')

if __name__ == "__main__":
    channel_id = 'UCUBNKEKMaWE-gkUZ2ptvyuQ'  # Replace with the actual channel ID
    process_channel(channel_id)
